In [23]:
import torch
import random
from scipy.optimize import linear_sum_assignment


In [1]:
row_ind, col_ind = [ 3, 57, 68, 94], [1, 2, 0, 3]

In [98]:
# outputs are the predicted bounding boxes and labels in the format ([batch_size, num_queries, 4], [batch_size, num_queries, num_classes])
# targets are the ground truth bounding boxes in the format [[batch_size, random, 4]] and labels in the format [[batch_size, num_classes]]
batch_size = 4
num_queries = 100
num_classes = 1
pred_boxes, pred_labels = torch.rand(batch_size, num_queries, 4), torch.rand(batch_size, num_queries, num_classes)
n_true = [random.randint(4, 95) for _ in range(batch_size)]
true_boxes = [torch.tensor(torch.rand(n_true[i], 4), dtype=torch.float32) for i in range(batch_size)]
true_labels = [torch.tensor([1]*n_true[i], dtype=torch.float32) for i in range(batch_size)]

C:\Users\rober\AppData\Local\Temp\ipykernel_19252\3852864391.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  true_boxes = [torch.tensor(torch.rand(n_true[i], 4), dtype=torch.float32) for i in range(batch_size)]


In [99]:
def hungarian(true, pred):
    true_boxes, true_labels = true
    boxes, labels = pred
    
    true_labels = true_labels.unsqueeze(1)
    
    # Calculamos el costo de emparejamiento entre las cajas predichas y verdaderas
    cost_boxes = torch.cdist(boxes, true_boxes, p=1)
    
    # Calculamos el costo de emparejamiento entre las etiquetas predichas y verdaderas
    cost_labels = torch.cdist(labels, true_labels, p=1)
    
    # Combinamos los costos
    cost = cost_boxes + cost_labels
    
    # Resolvemos el problema de asignación lineal
    row_ind, col_ind = linear_sum_assignment(cost.cpu().detach().numpy())
    
    return row_ind, col_ind

In [100]:
for (n), (pred_box, pred_label, true_box, true_label) in enumerate(zip(pred_boxes, pred_labels, true_boxes, true_labels)):
    row_ind, col_ind = hungarian((true_box, true_label), (pred_box, pred_label))
    true_boxes[n] = true_box[col_ind]
    true_labels[n] = true_label[col_ind]
    pred_boxes[n] = torch.cat((pred_box[row_ind], torch.zeros(num_queries - len(row_ind), 4)), dim=0)
    pred_labels[n] = torch.cat((pred_label[row_ind], torch.zeros(num_queries - len(row_ind), 1)), dim=0)

In [95]:
torch.all(pred_boxes == 0)

tensor(False)

In [104]:
pred_boxes = pred_boxes.flatten(0, 1)

In [108]:
pred_boxes[~torch.all(pred_boxes == 0, dim=1)]

tensor([[0.0221, 0.9547, 0.7671, 0.3143],
        [0.7985, 0.8174, 0.5870, 0.4615],
        [0.7599, 0.6510, 0.6496, 0.1845],
        [0.8678, 0.8971, 0.3087, 0.3714],
        [0.1934, 0.1493, 0.0432, 0.3627],
        [0.0846, 0.8856, 0.0831, 0.9489],
        [0.2258, 0.6916, 0.1412, 0.2136],
        [0.2991, 0.8308, 0.6685, 0.5517],
        [0.6066, 0.9706, 0.1884, 0.0564],
        [0.5314, 0.2056, 0.8985, 0.7409],
        [0.7055, 0.7111, 0.7041, 0.8047],
        [0.1334, 0.7800, 0.5335, 0.0076],
        [0.5338, 0.6305, 0.9066, 0.5470],
        [0.0943, 0.5216, 0.1501, 0.1656],
        [0.2092, 0.8585, 0.6053, 0.8748],
        [0.8064, 0.9187, 0.4954, 0.0821],
        [0.6137, 0.3807, 0.8499, 0.6924],
        [0.4523, 0.8538, 0.5355, 0.7502],
        [0.1406, 0.9251, 0.3245, 0.4903],
        [0.7693, 0.0052, 0.1557, 0.7456],
        [0.7634, 0.5426, 0.8675, 0.0739],
        [0.1088, 0.1245, 0.9854, 0.9446],
        [0.3437, 0.3496, 0.3123, 0.8982],
        [0.2084, 0.2028, 0.4103, 0

In [93]:
# remove from tensor
pred_boxes = pred_boxes.flatten(0, 1)
pred_labels = pred_labels.flatten(0, 1)

# remove zero rows
pred_boxes = pred_boxes[~torch.all(pred_boxes == 0, dim=1)]

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [86]:
true_boxes = true_boxes[~torch.all(true_boxes == 0, dim=1)]


tensor([[1.0134e-01, 3.9940e-01, 4.3783e-01, 8.2689e-01],
        [6.1661e-01, 9.3370e-01, 8.4658e-01, 8.6449e-01],
        [1.7256e-01, 5.7225e-01, 8.4777e-01, 9.0222e-01],
        [9.9274e-01, 3.6898e-02, 8.4661e-01, 8.4197e-01],
        [1.4833e-01, 4.4053e-01, 6.5149e-01, 2.0026e-01],
        [2.4150e-01, 8.9178e-01, 9.7699e-02, 6.4370e-01],
        [4.5470e-01, 9.0499e-02, 1.9406e-01, 3.7702e-01],
        [4.6146e-01, 2.7627e-01, 3.8143e-01, 2.4341e-01],
        [9.6464e-01, 9.7269e-01, 2.9742e-01, 8.7217e-01],
        [8.5463e-01, 3.2303e-01, 1.7339e-01, 4.2945e-01],
        [2.5283e-01, 5.0943e-01, 8.2694e-01, 5.6011e-01],
        [6.7699e-01, 9.7552e-01, 8.8453e-01, 2.9549e-01],
        [6.6048e-01, 6.3125e-01, 4.3162e-01, 9.8390e-01],
        [8.6755e-01, 2.4419e-01, 7.6131e-01, 1.7953e-01],
        [7.6937e-01, 7.5233e-02, 6.8938e-01, 1.0732e-01],
        [7.2804e-01, 5.2919e-01, 1.2710e-01, 2.2019e-01],
        [3.0379e-02, 3.7417e-01, 1.8657e-01, 4.9309e-01],
        [9.693

In [82]:
pred_boxes

tensor([[[0.1013, 0.3994, 0.4378, 0.8269],
         [0.6166, 0.9337, 0.8466, 0.8645],
         [0.1726, 0.5722, 0.8478, 0.9022],
         ...,
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000]],

        [[0.3764, 0.9015, 0.0495, 0.7593],
         [0.2775, 0.9334, 0.3636, 0.8425],
         [0.0796, 0.5180, 0.0703, 0.5655],
         ...,
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000]],

        [[0.5194, 0.0535, 0.7860, 0.7460],
         [0.8135, 0.0248, 0.9671, 0.3418],
         [0.3295, 0.9884, 0.4461, 0.8420],
         ...,
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000]],

        [[0.5984, 0.1600, 0.7006, 0.6507],
         [0.2053, 0.3442, 0.1800, 0.1763],
         [0.9540, 0.8669, 0.9557, 0.9204],
         ...,
         [0.0000, 0.0000, 0.0000, 0